# Project 2

## **Importing Necessary Libraries**


In [27]:
# Import necessary libraries for data manipulation, visualization, and machine learning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import joblib
import string
import math
import time

#Libraries for Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import confusion_matrix
from collections import Counter

# Sklearn libraries for machine learning and text processing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold, cross_validate, validation_curve, learning_curve
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.utils.class_weight import compute_class_weight

from sklearn import metrics


# NLTK libraries for text processing (lemmatization, stemming, stopwords, POS tagging)
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize

# Set up visualization
%matplotlib inline
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

# Download necessary NLTK resources for text processing
nltk.download('wordnet')  # WordNet for lemmatization
nltk.download('omw-1.4')  # Open Multilingual Wordnet
nltk.download('punkt')  # Tokenizer
nltk.download('punkt_tab')
nltk.download('stopwords')  # Stopwords for text cleaning
nltk.download('averaged_perceptron_tagger')  # POS tagger for part-of-speech tagging
nltk.download('averaged_perceptron_tagger_eng')  # Additional tagger
nltk.download('tagsets_json')  # Tagset resource

print("\n All libraries imported successfully!")


 All libraries imported successfully!


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagse

## Import Data

In [2]:
from google.colab import files
uploaded = files.upload()


Saving AI_vs_huam_train_dataset.xlsx to AI_vs_huam_train_dataset.xlsx
Saving Final_test_data.csv to Final_test_data.csv


## 💾 Reading the dataset into pandas

In [3]:
!pip install openpyxl
import pandas as pd
df_train = pd.read_excel('AI_vs_huam_train_dataset.xlsx')
df_test = pd.read_csv('Final_test_data.csv')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.3 MB/s eta 0:00:00


In [4]:
print("Dataset Information:")
print(f"Training data shape: {df_train.shape}")
print(f"Test data shape: {df_test.shape}")
print(f"Columns: {df_train.columns.tolist()}")

# Show first few rows
print("\nFirst 5 rows of training data:")
print(df_train.head())

Dataset Information:
Training data shape: (3728, 2)
Test data shape: (869, 2)
Columns: ['essay', 'label']

First 5 rows of training data:
                                               essay  label
0  International sports events require the most w...      0
1  Globalisation has become a significant aspect ...      0
2  There is an ever-increasing number of bullying...      0
3  It is commonly believed, that companies should...      0
4  Despite knowing about the adverse effects of c...      0


In [ ]:
df_train.info()
df_train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3728 entries, 0 to 3727
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   essay   3728 non-null   object
 1   label   3728 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 58.4+ KB


,label
count,3728.000000
mean,0.500000
std,0.500067
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [ ]:
print('Class Distribution')
print(df_train["label"].value_counts)

Class Distribution
<bound method IndexOpsMixin.value_counts of 0       0
1       0
2       0
3       0
4       0
       ..
3723    1
3724    1
3725    1
3726    1
3727    1
Name: label, Length: 3728, dtype: int64>


In [ ]:
print("Missing values in test dataset")
print(df_test.isnull().sum())

Missing values in test dataset
essay      0
label    869
dtype: int64


In [ ]:
# Samples from each category
print("Random Sample Human-written texts:")
print(df_train[df_train['label'] == 0]['essay'].sample(3, random_state=42).tolist())

print("\nRandom Sample AI-generated texts:")
print(df_train[df_train['label'] == 1]['essay'].sample(3, random_state=42).tolist())


Random Sample Human-written texts:
['Childhood memories are one of the most memorable experiences for many children. Some of people argue that these memories are highly influenced the children on their future life. In contrary many said that experiences occurred during teenage perod ‡πâhave much more impact to their future. In this essay will disscuss on both point of view and will show some supportive opinions that experiences during the time at school are highly impact futher development in the future for many teenagers.\nOn one hand, it is the fact that experiences or events happend during their childhood or before they starting to go to school can highly attached to their memories. For example, children would be able to remember the situation where they felt the most insecure or scared, for example, some scary movies or fear of some animals. In which will be further more becomes their wound in their heart. However, these experiences can be erradicated or overcame after in their lif

## ** Text Pre-processing**

> Our data is in text format, however the classification models need some sort of numerical feature vector in order to perform the classification task. There  many methods to convert a corpus to a vector format. The simplest is the `bag-of-words` approach, where each unique word in a text will be represented by one number.

> Here, we will split the essay into its individual words and return a list. We'll also remove very common words, ('the', 'a', etc..), which we often call as "Stop Words". To do this, we will take advantage of the `NLTK` library, a standard library in Python for processing text and has a lot of useful features.


In [11]:
#  Advanced Text Preprocessing with Lemmatization

class TextPreprocessor:
    """
    Advanced text preprocessing class with lemmatization
    This will clean text better than basic preprocessing
    """

    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        # Add custom stop words
        self.stop_words.update(['u', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure'])

    def get_wordnet_pos(self, word):
        """Convert POS tag to format accepted by WordNet lemmatizer"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)

    def clean_text(self, text):
        """Basic text cleaning"""
        if not isinstance(text, str):
            return ""

        # Converting to lowercase
        text = text.lower()

        # Removing URLs, emails, HTML tags
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        text = re.sub(r'\S+@\S+', '', text)
        text = re.sub(r'<.*?>', '', text)

        # Removing special characters but keep spaces
        text = re.sub(r'[^a-zA-Z\s]', '', text)

        # Removing extra whitespace
        text = ' '.join(text.split())

        return text

    def lemmatize_text(self, text):
        """Apply lemmatization with POS tagging"""
        tokens = word_tokenize(text)

        lemmatized_tokens = []
        for token in tokens:
            if token not in self.stop_words and len(token) > 2:
                # Gettiing POS tag and lemmatize
                pos_tag = self.get_wordnet_pos(token)
                lemmatized_token = self.lemmatizer.lemmatize(token, pos_tag)
                lemmatized_tokens.append(lemmatized_token)

        return ' '.join(lemmatized_tokens)

    def preprocess(self, text):
        """Complete preprocessing pipeline"""
        cleaned_text = self.clean_text(text)
        lemmatized_text = self.lemmatize_text(cleaned_text)
        return lemmatized_text

print("Text preprocessing class created!")

Text preprocessing class created!


In [12]:
# Applying Text Preprocessing to the "essay" column

# Initialize the preprocessor
preprocessor = TextPreprocessor()

print("Applying advanced text preprocessing with lemmatization...")

# Apply preprocessing to training data
df_train['clean_essay'] = df_train['essay'].apply(preprocessor.preprocess)

# Apply preprocessing to test data
df_test['clean_essay'] = df_test['essay'].apply(preprocessor.preprocess)

print("Text preprocessing completed!")

Applying advanced text preprocessing with lemmatization...
Text preprocessing completed!


In [13]:
# Show examples of preprocessing
print("\nExamples of text preprocessing:")
print("\nOriginal:", df_train['essay'].iloc[0])
print("\nProcessed:", df_train['clean_essay'].iloc[0])
print()
print("Original:", df_train['essay'].iloc[5])
print("\nProcessed:", df_train['clean_essay'].iloc[5])


Examples of text preprocessing:

Original: International sports events require the most well-trained athletes for each country, in order to achieve this goal countries make an effort to build infrastructure designed to train top athletes. Although this policy can indeed make fewer sports facilities for ordinary people, investing in the best athletes is vital to develop competitive sports performances in each country.
On the one hand, building specific infrastructure for the best athletes is crucial in order to get better results at international sports events such as The Olympics or the World Cup. The importance of getting better results is that it creates awareness of the importance of sports in society and motivates more people to do a sport. In this way, investing in these developments can help countries to develop an integral sport policy that can benefit everyone.
On the other hand, one can argue that a negative effect could be that less infrastructure is built for the rest of th

In [14]:
# Define features and target
X = df_train['clean_essay']
y = df_train['label']

print(f"Features (X): {len(X)} input essay")
print(f"Target (y): {len(y)} labels")

Features (X): 3728 input essay
Target (y): 3728 labels


In [15]:
# Split Data for Training and Validation Sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)    # stratify=y Maintain label distribution

In [16]:
print("Data split completed:")
print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"Training label distribution: {np.bincount(y_train)}")
print(f"Validation label distribution: {np.bincount(y_val)}")

Data split completed:
Training samples: 2982
Validation samples: 746
Training label distribution: [1491 1491]
Validation label distribution: [373 373]


## **4. Representing Text as Numerical Data using TF-IDF vectorization**

Before training our machine learning models, we need to convert the text data into a numerical format that the algorithms can understand. This process is called **text vectorization**. We will explore and use Tf-IDF text vectorization techniques:

- **TF-IDF (Term Frequency-Inverse Document Frequency):** This method weighs words based on their importance in a document relative to the entire corpus.


In [17]:
# Transform Text to Numbers Using TF-IDF
# Create TF-IDF vectorizer with optimal parameters
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,        # Keep top 5000 most important words
    ngram_range=(1, 2),       # Single word and word pairs
    min_df=2,
    max_df=0.95,              # Ignore words that appear in >95% of documents
    stop_words='english'      # Remove common English stop words
)

print("TF-IDF Vectorizer created with optimal parameters:")
print(f"- Max features: 5000")
print(f"- N-gram range: (1, 2) - unigrams and bigrams")
print(f"- Min document frequency: 2")
print(f"- Max document frequency: 0.95")

TF-IDF Vectorizer created with optimal parameters:
- Max features: 5000
- N-gram range: (1, 2) - unigrams and bigrams
- Min document frequency: 2
- Max document frequency: 0.95


In [18]:
# Fit TF-IDF on training data and transform both sets
print("Converting text to numerical features using TF-IDF...")

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

print("TF-IDF transformation completed!")
print(f"Training matrix shape: {X_train_tfidf.shape}")
print(f"Validation matrix shape: {X_val_tfidf.shape}")

# Show some feature names
feature_names = tfidf_vectorizer.get_feature_names_out()
print(f"Total features created: {len(feature_names)}")
print(f"Sample features: {list(feature_names[:15])}")

Converting text to numerical features using TF-IDF...
TF-IDF transformation completed!
Training matrix shape: (2982, 5000)
Validation matrix shape: (746, 5000)
Total features created: 5000
Sample features: ['abandon', 'ability', 'ability make', 'ability risk', 'able', 'able make', 'able risk', 'abroad', 'absence', 'absolute', 'absolutely', 'abuse', 'academic', 'academic ability', 'academically']


## **5. Machine Learning Model Training and Testing**

We will train and test three machine learning algorithms

- SVM
- Decision Trees
- AdaBoostClassifier

### SVM Model (Using GridSearchCV)

In [ ]:
svm_pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', SVC(probability=True, random_state=42))
])

# The required parameter grid
svm_param_grid = {
    'vectorizer__max_features': [1000, 5000, 10000],
    'vectorizer__ngram_range': [(1,1), (1,2), (1,3)],
    'classifier__C': [0.1, 1, 10, 100],
    'classifier__kernel': ['linear', 'rbf'],
    'classifier__gamma': ['scale', 'auto', 0.001, 0.01]  # Only for rbf kernel
}

# Setting up GridSearchCV with 5-fold cross-validation
svm_grid = GridSearchCV(
    svm_pipeline,
    param_grid=svm_param_grid,
    cv=5,
    scoring='accuracy',
    verbose=3,
    n_jobs=-1,
    refit = True,
)

# Fitting grid search
svm_grid.fit(X_train, y_train)

# Best parameters
print("Best SVM parameters: ", svm_grid.best_params_)

# Best accuracy
print("Best SVM accuracy: ", svm_grid.best_score_)


Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Best SVM parameters:  {'classifier__C': 10, 'classifier__gamma': 'scale', 'classifier__kernel': 'linear', 'vectorizer__max_features': 5000, 'vectorizer__ngram_range': (1, 3)}
Best SVM accuracy:  0.9731684147808393


### Decision Trees Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier


dt_pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Parameter grid for Decision Tree
dt_param_grid = {
    'vectorizer__max_features': [1000, 5000, 10000],
    'vectorizer__ngram_range': [(1,1), (1,2), (1,3)],
    'classifier__criterion': ['gini', 'entropy'],
    'classifier__max_depth': [10, 20, 30, None],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

# Setting up GridSearchCV with 5-fold CV
dt_grid = GridSearchCV(
    estimator=dt_pipeline,
    param_grid=dt_param_grid,
    cv=5,
    scoring='accuracy',
    verbose=3,
    n_jobs=-1,
    refit = True
)

# Training the GridSearchCV
dt_grid.fit(X_train, y_train)

# Output results
print("Best Decision Tree Parameters:", dt_grid.best_params_)
print("Best Decision Tree Accuracy:", dt_grid.best_score_)

Fitting 5 folds for each of 648 candidates, totalling 3240 fits
Best Decision Tree Parameters: {'classifier__criterion': 'gini', 'classifier__max_depth': 10, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 10, 'vectorizer__max_features': 5000, 'vectorizer__ngram_range': (1, 1)}
Best Decision Tree Accuracy: 0.8708818139916585


### AdaBoost Classifier

In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

pipe = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', AdaBoostClassifier(
        estimator=DecisionTreeClassifier(max_depth=1),
        random_state=42
    ))
])

param_grid = {
    'clf__n_estimators':         [50, 100, 200],
    'clf__learning_rate':        [0.01, 0.1, 1.0],
    'clf__estimator__max_depth': [1, 2, 3]
}

grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)     # train

print("Best params:", grid.best_params_)
print("CV score:", grid.best_score_)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best params: {'clf__estimator__max_depth': 3, 'clf__learning_rate': 1.0, 'clf__n_estimators': 200}
CV score: 0.9664665609928837


### Cross-Validation and Confusion Matrices for all the models

#### SVM Model

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix

# 1. Grab the refit pipeline
svm_best_pipeline = svm_grid.best_estimator_

# 2. Cross-validate accuracy
svm_cv_scores = cross_val_score(
    svm_best_pipeline,
    X_train,
    y_train,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)
print("SVM CV Accuracy: %0.3f (+/- %0.3f)" %
      (svm_cv_scores.mean(), svm_cv_scores.std()))

# 3. Per-fold predictions to build a classification report & confusion matrix
y_pred = cross_val_predict(
    svm_best_pipeline,
    X_train,
    y_train,
    cv=5,
    n_jobs=-1
)

print("\nClassification Report:\n", classification_report(y_train, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred))

SVM CV Accuracy: 0.973 (+/- 0.007)

Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.97      0.97      1491
           1       0.97      0.98      0.97      1491

    accuracy                           0.97      2982
   macro avg       0.97      0.97      0.97      2982
weighted avg       0.97      0.97      0.97      2982

Confusion Matrix:
 [[1446   45]
 [  35 1456]]


### Decision Trees Model

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix

# 1. Grab the refit pipeline
dt_best_pipeline = dt_grid.best_estimator_

# 2. Cross-validate accuracy
dt_cv_scores = cross_val_score(
    dt_best_pipeline,
    X_train,
    y_train,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)
print("DT CV Accuracy: %0.3f (+/- %0.3f)" %
      (dt_cv_scores.mean(), dt_cv_scores.std()))

# 3. Per-fold predictions to build a classification report & confusion matrix
y_pred = cross_val_predict(
    dt_best_pipeline,
    X_train,
    y_train,
    cv=5,
    n_jobs=-1
)

print("\nClassification Report:\n", classification_report(y_train, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred))

DT CV Accuracy: 0.871 (+/- 0.017)

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.89      0.87      1491
           1       0.88      0.86      0.87      1491

    accuracy                           0.87      2982
   macro avg       0.87      0.87      0.87      2982
weighted avg       0.87      0.87      0.87      2982

Confusion Matrix:
 [[1320  171]
 [ 214 1277]]


#### AdaBoost Classifier Model

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix

# 1. Grab the refit pipeline
ada_best_pipeline = grid.best_estimator_

# 2. Cross-validate accuracy
ada_cv_scores = cross_val_score(
    ada_best_pipeline,
    X_train,
    y_train,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)
print("AdaBoost CV Accuracy: %0.3f (+/- %0.3f)" %
      (ada_cv_scores.mean(), ada_cv_scores.std()))

# 3. Get per-fold predictions to build a classification report & confusion matrix
y_pred = cross_val_predict(
    ada_best_pipeline,
    X_train,
    y_train,
    cv=5,
    n_jobs=-1
)

print("\nClassification Report:\n", classification_report(y_train, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred))


AdaBoost CV Accuracy: 0.966 (+/- 0.005)

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.97      0.97      1491
           1       0.97      0.97      0.97      1491

    accuracy                           0.97      2982
   macro avg       0.97      0.97      0.97      2982
weighted avg       0.97      0.97      0.97      2982

Confusion Matrix:
 [[1443   48]
 [  52 1439]]


## **Deep Learning Models**

In [22]:
#Loading the dataset again due to some problems
train_data = pd.read_excel('AI_vs_huam_train_dataset.xlsx')
test_data = pd.read_csv('Final_test_data.csv')

# --------------------
# Tokenizer + Vocab
# --------------------
def tokenize(text):
    return text.lower().split()

counter = Counter()
for essay in train_data['essay']:
    counter.update(tokenize(essay))

vocab = {word: idx+2 for idx, (word, _) in enumerate(counter.items())}
vocab['<PAD>'] = 0
vocab['<UNK>'] = 1

with open("vocab.pkl", "wb") as f:
    import pickle
    pickle.dump(vocab, f)

# --------------------
# Encode function
# --------------------
def encode(text):
    return [vocab.get(token, 1) for token in tokenize(text)]
 # --------------------
# Dataset + Collate
# --------------------
class EssayDataset(Dataset):
    def __init__(self, essays, labels=None):
        self.essays = [torch.tensor(encode(e)) for e in essays]
        self.labels = labels

    def __len__(self):
        return len(self.essays)

    def __getitem__(self, idx):
        if self.labels is not None:
            return self.essays[idx], self.labels[idx]
        else:
            return self.essays[idx]

def collate_batch(batch):
    if isinstance(batch[0], tuple):
        essays, labels = zip(*batch)
        essays_padded = pad_sequence(essays, batch_first=True, padding_value=0)
        return essays_padded, torch.tensor(labels)
    else:
        essays_padded = pad_sequence(batch, batch_first=True, padding_value=0)
        return essays_padded

# --------------------
# DataLoaders
# --------------------
y_train = train_data['label'].tolist()
dataset = EssayDataset(train_data['essay'], y_train)
train_len = int(0.8 * len(dataset))
val_len = len(dataset) - train_len
train_dataset, val_dataset = random_split(dataset, [train_len, val_len])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_batch)

### LSTM Model

In [23]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=100, hidden_dim=256, output_dim=2, dropout=0.5):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.lstm(embedded)
        out = self.dropout(hidden[-1])
        return self.fc(out)

### CNN Model

In [24]:
class CNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=100, num_filters=128, filter_sizes=[3,4,5], output_dim=2, dropout=0.5):
        super(CNNClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (fs, embed_dim)) for fs in filter_sizes
        ])
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(num_filters * len(filter_sizes), output_dim)
    def forward(self, x):
        embedded = self.embedding(x).unsqueeze(1)
        conved = [torch.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        pooled = [torch.max(c, dim=2)[0] for c in conved]
        cat = torch.cat(pooled, dim=1)
        out = self.dropout(cat)
        return self.fc(out)


### RNN Model

In [25]:
class RNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=100, hidden_dim=512, output_dim=2, dropout=0.5, bidirectional=True):
        super(RNNClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True, bidirectional=bidirectional)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
    def forward(self, x):
        embedded = self.embedding(x)
        _, hidden = self.rnn(embedded)
        if isinstance(hidden, tuple):  # just safety if ever swapped
            hidden = hidden[0]
        if self.rnn.bidirectional:
            out = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        else:
            out = hidden[-1]
        out = self.dropout(out)
        return self.fc(out)


#### Training the models

Now, Training the models

In [28]:
# Weighted loss
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
weights = torch.tensor(class_weights, dtype=torch.float)
criterion = nn.CrossEntropyLoss(weight=weights)

# --------------------
# Train function (generic)
# --------------------
def train_model(model, name, optimizer, num_epochs=20, patience=3):
    best_val_acc = 0
    trigger_times = 0
    for epoch in range(num_epochs):
        model.train()
        for x_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x_batch, y_batch in val_loader:
                outputs = model(x_batch)
                preds = torch.argmax(outputs, dim=1)
                correct += (preds == y_batch).sum().item()
                total += y_batch.size(0)
        val_acc = correct / total
        print(f"{name} - Epoch {epoch+1}, Val Accuracy: {val_acc:.4f}")

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            trigger_times = 0
            torch.save(model.state_dict(), f"best_{name}_model.pt")
        else:
            trigger_times += 1
            if trigger_times >= patience:
                print("Early stopping triggered.")
                break
    print(f"Best Validation Accuracy for {name}: {best_val_acc:.4f}")

Training LSTM

In [29]:
lstm_model = LSTMClassifier(len(vocab))
optimizer = optim.Adam(lstm_model.parameters(), lr=0.0005)
train_model(lstm_model, "lstm", optimizer)

lstm - Epoch 1, Val Accuracy: 0.5255
lstm - Epoch 2, Val Accuracy: 0.5228
lstm - Epoch 3, Val Accuracy: 0.5295
lstm - Epoch 4, Val Accuracy: 0.5282
lstm - Epoch 5, Val Accuracy: 0.5241
lstm - Epoch 6, Val Accuracy: 0.5308
lstm - Epoch 7, Val Accuracy: 0.6206
lstm - Epoch 8, Val Accuracy: 0.5912
lstm - Epoch 9, Val Accuracy: 0.5670
lstm - Epoch 10, Val Accuracy: 0.7534
lstm - Epoch 11, Val Accuracy: 0.8298
lstm - Epoch 12, Val Accuracy: 0.9115
lstm - Epoch 13, Val Accuracy: 0.9491
lstm - Epoch 14, Val Accuracy: 0.9477
lstm - Epoch 15, Val Accuracy: 0.9665
lstm - Epoch 16, Val Accuracy: 0.9611
lstm - Epoch 17, Val Accuracy: 0.9571
lstm - Epoch 18, Val Accuracy: 0.9625
Early stopping triggered.
Best Validation Accuracy for lstm: 0.9665


Training CNN

In [30]:
cnn_model = CNNClassifier(len(vocab))
optimizer = optim.Adam(cnn_model.parameters(), lr=0.0005)
train_model(cnn_model, "cnn", optimizer)

cnn - Epoch 1, Val Accuracy: 0.9223
cnn - Epoch 2, Val Accuracy: 0.9290
cnn - Epoch 3, Val Accuracy: 0.9517
cnn - Epoch 4, Val Accuracy: 0.9531
cnn - Epoch 5, Val Accuracy: 0.9611
cnn - Epoch 6, Val Accuracy: 0.9611
cnn - Epoch 7, Val Accuracy: 0.9665
cnn - Epoch 8, Val Accuracy: 0.9692
cnn - Epoch 9, Val Accuracy: 0.9705
cnn - Epoch 10, Val Accuracy: 0.9732
cnn - Epoch 11, Val Accuracy: 0.9705
cnn - Epoch 12, Val Accuracy: 0.9745
cnn - Epoch 13, Val Accuracy: 0.9692
cnn - Epoch 14, Val Accuracy: 0.9705
cnn - Epoch 15, Val Accuracy: 0.9705
Early stopping triggered.
Best Validation Accuracy for cnn: 0.9745


Traning RNN

In [31]:
rnn_model = RNNClassifier(len(vocab), embed_dim=100, hidden_dim=512, output_dim=2, dropout=0.5, bidirectional=True)
optimizer = optim.Adam(rnn_model.parameters(), lr=0.0005)
train_model(rnn_model, "rnn", optimizer, num_epochs=40, patience=5)

rnn - Epoch 1, Val Accuracy: 0.6676
rnn - Epoch 2, Val Accuracy: 0.7426
rnn - Epoch 3, Val Accuracy: 0.7828
rnn - Epoch 4, Val Accuracy: 0.5228
rnn - Epoch 5, Val Accuracy: 0.5080
rnn - Epoch 6, Val Accuracy: 0.5416
rnn - Epoch 7, Val Accuracy: 0.5322
rnn - Epoch 8, Val Accuracy: 0.5308
Early stopping triggered.
Best Validation Accuracy for rnn: 0.7828


### Saving the models

In [ ]:
torch.save(rnn_model.state_dict(), "rnn_model.pkl")
torch.save(lstm_model.state_dict(), "lstm_model.pkl")
torch.save(cnn_model.state_dict(), "cnn_model.pkl")

## **6. Model saving for Streamlit deployment**

In [ ]:
pipeline_predictions = svm_pipeline.predict(X_val)
pipeline_accuracy = metrics.accuracy_score(y_val, pipeline_predictions)

print("Final Pipeline Results:")
print(f"Validation accuracy: {pipeline_accuracy:.4f}")

# Show detailed classification report
print("\nDetailed Classification Report:")
target_names = ['Human', 'AI']
classification_report = metrics.classification_report(y_val, pipeline_predictions, target_names=target_names)
print(classification_report)

Final Pipeline Results:
Validation accuracy: 0.9692

Detailed Classification Report:
              precision    recall  f1-score   support

       Human       0.97      0.97      0.97       373
          AI       0.97      0.97      0.97       373

    accuracy                           0.97       746
   macro avg       0.97      0.97      0.97       746
weighted avg       0.97      0.97      0.97       746



In [ ]:
#Save Models for Streamlit Deployment

print("Saving models for Streamlit deployment...")

# Save the complete pipeline (includes TF-IDF + classifier)
pipeline_filename = 'human_ai_pipeline.pkl'
joblib.dump(svm_best_pipeline, pipeline_filename)

# Also save individual components for flexibility
tfidf_filename = 'tfidf_vectorizer.pkl'

joblib.dump(tfidf_vectorizer, tfidf_filename)

print("Models saved successfully!")
print(f"Complete pipeline: {pipeline_filename}")
print(f"TF-IDF vectorizer: {tfidf_filename}")

Saving models for Streamlit deployment...
Models saved successfully!
Complete pipeline: human_ai_pipeline.pkl
TF-IDF vectorizer: tfidf_vectorizer.pkl


#### 2.  For Decision Tree classifier

In [ ]:
# Test classifier performance
dt_predictions = dt_best_pipeline.predict(X_val)
dt_accuracy = metrics.accuracy_score(y_val, dt_predictions)

print(f"\Decision Tree Results:")
print(f"Validation accuracy: {dt_accuracy:.4f}")

# Show detailed classification report
print("\nDetailed Classification Report - Decision Trees:")
target_names = ['Human', 'AI']
dt_classification_report = metrics.classification_report(y_val, dt_predictions, target_names=target_names)
print(dt_classification_report)

# Save DT Classifier
print("Saving Decision Trees Classifier")
print("="*60)

# Save the DT classifier
dt_model_filename = 'decision_tree_model.pkl'
print("Saving Decision Tree classifier...")
joblib.dump(dt_best_pipeline, dt_model_filename)

print(f"\n Decision Trees classifier saved as: {dt_model_filename}")

\Decision Tree Results:
Validation accuracy: 0.8606

Detailed Classification Report - Decision Trees:
              precision    recall  f1-score   support

       Human       0.83      0.90      0.87       373
          AI       0.89      0.82      0.85       373

    accuracy                           0.86       746
   macro avg       0.86      0.86      0.86       746
weighted avg       0.86      0.86      0.86       746

Saving Decision Trees Classifier
Saving Decision Tree classifier...

 Decision Trees classifier saved as: decision_tree_model.pkl


#### 3. For AdaBoost Classifier


In [ ]:
ada_predictions = ada_best_pipeline.predict(X_val)
ada_accuracy = metrics.accuracy_score(y_val, ada_predictions)

print(f"\AdaBoost Results:")
print(f"Validation accuracy: {ada_accuracy:.4f}")

# Showing detailed classification report
print("\nDetailed Classification Report - AdaBoost Classifier:")
target_names = ['Human', 'AI']
ada_classification_report = metrics.classification_report(y_val, ada_predictions, target_names=target_names)
print(ada_classification_report)

# Saving The AdaBoost Classifier
print("Saving AdaBoost Classifier")
print("="*60)

# Saving the AdaBoost classifier
ada_model_filename = 'adaBoost_model.pkl'
print("Saving Decision Tree classifier...")
joblib.dump(ada_best_pipeline, ada_model_filename)

print(f"\n AdaBoost Classifier saved as: {ada_model_filename}")


\AdaBoost Results:
Validation accuracy: 0.9705

Detailed Classification Report - AdaBoost Classifier:
              precision    recall  f1-score   support

       Human       0.97      0.97      0.97       373
          AI       0.97      0.97      0.97       373

    accuracy                           0.97       746
   macro avg       0.97      0.97      0.97       746
weighted avg       0.97      0.97      0.97       746

Saving AdaBoost Classifier
Saving Decision Tree classifier...

 AdaBoost Classifier saved as: adaBoost_model.pkl


## **Test the Test Data on the Saved Models**

In [ ]:
X_test = df_test['essay']
Y_test = df_test['label']
test_essay = X_test.head()
print(test_essay)

0    Enjoyment means entertainment, satisfaction of...
1    Before to start with my opinion of the topic i...
2    Thats a really really good subject, but acctul...
3    I disagree with the statement saying that it i...
4    It is undeniable that advertisements play a si...
Name: essay, dtype: object


### **Loading TF-IDF Vectorizer and Saved ML models**

In [ ]:
# Load existing TF-IDF vectorizer
print("Loading existing TF-IDF vectorizer...")
tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')
print("TF-IDF vectorizer loaded successfully!")

# Load the saved DT classifier
print("Loading saved DT classifier pipeline...")
loaded_dt_classifier = joblib.load('decision_tree_model.pkl')
print("✅ Decision Tree loaded successfully!")
dt_available = True

# Load the saved SVM Classifier for comparison
print("Loading saved SVM pipeline...")
loaded_pipeline_svm = joblib.load('human_ai_pipeline.pkl')
svm_available = True
print("✅ SVM pipeline loaded successfully!")

# Load the saved AdaBoost Classifier
print("Loading saved AdaBoost classifier pipeline...")
loaded_ada_classifier = joblib.load('adaBoost_model.pkl')
print("✅ AdaBoost Classifier loaded successfully!")
ada_available = True



Loading existing TF-IDF vectorizer...
TF-IDF vectorizer loaded successfully!
Loading saved DT classifier pipeline...
✅ Decision Tree loaded successfully!
Loading saved SVM pipeline...
✅ SVM pipeline loaded successfully!
Loading saved AdaBoost classifier pipeline...
✅ AdaBoost Classifier loaded successfully!


#### Testing

In [ ]:
if dt_available:
  print("Testing Decision Trees predictions...")

  for i, essay in enumerate(test_essay, 1):
    print(f"{i}. Test: {essay}")
    dt_prediction = loaded_dt_classifier.predict(X_test)

    print()
else:
  print("Cannot test DT - model not found")

Testing Decision Trees predictions...
1. Test: Enjoyment means entertainment, satisfaction of one's soul and containment with the happenings in life. I would like to disagree with this statement. This is because entertainment would mean differently to different people. Entertainment depends on various factors like the person's attitude and his notion or perspective on entertainment.
   The younger generation tends to enjoy themselves by going to parties, meeting new people, making new friends and going for long drives. The way a youngster enjoys himself or herself depends on the attitude and nature of the individual. Some people might like to go to parties for entertainment while others might like to watch a movie.
   On the other hand the older people might also do the same things that youngsters do or they may engage in activities considered as dull and boring by the youngsters. I would like to take examples of two famous personalities here who despite their age never complained of b

In [ ]:
if ada_available:
  print("Testing AdaBoost predictions...")

  for i, essay in enumerate(test_essay, 1):
    print(f"{i}. Test: {essay}")
    ada_prediction = loaded_ada_classifier.predict(X_test)

    print()
else:
  print("Cannot test AdaBoost - model not found")

Testing AdaBoost predictions...
1. Test: Enjoyment means entertainment, satisfaction of one's soul and containment with the happenings in life. I would like to disagree with this statement. This is because entertainment would mean differently to different people. Entertainment depends on various factors like the person's attitude and his notion or perspective on entertainment.
   The younger generation tends to enjoy themselves by going to parties, meeting new people, making new friends and going for long drives. The way a youngster enjoys himself or herself depends on the attitude and nature of the individual. Some people might like to go to parties for entertainment while others might like to watch a movie.
   On the other hand the older people might also do the same things that youngsters do or they may engage in activities considered as dull and boring by the youngsters. I would like to take examples of two famous personalities here who despite their age never complained of boredom

In [ ]:
#Test SVM Model
# Option 1: Use Pipeline

print("OPTION 1: Testing SVM Pipeline")
print("=" * 60)

# Load the saved pipeline
loaded_pipeline_svm = joblib.load(pipeline_filename)

for i, text in enumerate(test_essay, 1):
    # Use raw text - let pipeline handle all preprocessing
    prediction = loaded_pipeline_svm.predict([text])[0]
    probability = loaded_pipeline_svm.predict_proba([text])[0]

    classification = "Human" if prediction == 0 else "AI"
    confidence = max(probability)

    print(f"{i}. Text: {text}")
    print(f"\n   Prediction: {classification} (Confidence: {confidence:.3f})")
    print(f"\n  Probabilities: Human: {probability[0]:.3f}, AI: {probability[1]:.3f}")
    print()

print("\n" + "="*60)

OPTION 1: Testing SVM Pipeline
1. Text: Enjoyment means entertainment, satisfaction of one's soul and containment with the happenings in life. I would like to disagree with this statement. This is because entertainment would mean differently to different people. Entertainment depends on various factors like the person's attitude and his notion or perspective on entertainment.
   The younger generation tends to enjoy themselves by going to parties, meeting new people, making new friends and going for long drives. The way a youngster enjoys himself or herself depends on the attitude and nature of the individual. Some people might like to go to parties for entertainment while others might like to watch a movie.
   On the other hand the older people might also do the same things that youngsters do or they may engage in activities considered as dull and boring by the youngsters. I would like to take examples of two famous personalities here who despite their age never complained of boredom.

In [ ]:
# Compareing all Models Side by Side
# ========================================

if svm_available and (dt_available and ada_available):
    print("COMPARING All MODELS")
    print("=" * 70)

    agreements = 0
    disagreements = 0

    for i, text in enumerate(test_essay, 1):
        print(f"{i}. Text: {text}")

        # Test Decision Trees
        dt_prediction = loaded_dt_classifier.predict(X_test)[0]
        dt_probability = loaded_dt_classifier.predict_proba(X_test)[0]
        dt_classification = "Human" if dt_prediction == 0 else "AI"
        dt_confidence = max(dt_probability)

        # Test SVM
        if svm_available:
            svm_prediction = loaded_pipeline_svm.predict([text])[0]
            svm_probability = loaded_pipeline_svm.predict_proba([text])[0]

        svm_classification = "Human" if svm_prediction == 0 else "AI"
        svm_confidence = max(svm_probability)

        # Test AdaBoost
        if ada_available:
            ada_prediction = loaded_pipeline_svm.predict([text])[0]
            ada_probability = loaded_pipeline_svm.predict_proba([text])[0]
        ada_classification = "Human" if ada_prediction == 0 else "AI"
        ada_confidence = max(ada_probability)


        print(f"   Decision Tree: {dt_classification} (Confidence: {dt_confidence:.3f})")
        print(f"   SVM:   {svm_classification} (Confidence: {dt_confidence:.3f})")
        print(f"   ADaBoost: {dt_classification} (Confidence: {dt_confidence:.3f})")


        # Show agreement/disagreement
        if dt_classification == svm_classification == ada_classification:
            print(f"   Agreement:All models agree on {svm_classification}")
            agreements += 1
        else:
            print(f"   Disagreement: SVM: {svm_classification}, DT: {dt_classification}, ADA: {ada_classification}")
            disagreements += 1

        print()

COMPARING All MODELS
1. Text: Enjoyment means entertainment, satisfaction of one's soul and containment with the happenings in life. I would like to disagree with this statement. This is because entertainment would mean differently to different people. Entertainment depends on various factors like the person's attitude and his notion or perspective on entertainment.
   The younger generation tends to enjoy themselves by going to parties, meeting new people, making new friends and going for long drives. The way a youngster enjoys himself or herself depends on the attitude and nature of the individual. Some people might like to go to parties for entertainment while others might like to watch a movie.
   On the other hand the older people might also do the same things that youngsters do or they may engage in activities considered as dull and boring by the youngsters. I would like to take examples of two famous personalities here who despite their age never complained of boredom. Firstly, 

In [ ]:
# Summary statistics
total_tests = len(test_essay)
agreement_rate = (agreements / total_tests) * 100

print("=" * 70)
print("MODEL COMPARISON SUMMARY")
print("=" * 70)
print(f"Total tests: {total_tests}")
print(f"Agreements: {agreements}")
print(f"Disagreements: {disagreements}")
print(f"Agreement rate: {agreement_rate:.1f}%")

MODEL COMPARISON SUMMARY
Total tests: 5
Agreements: 4
Disagreements: 1
Agreement rate: 80.0%


In [1]:
# Model Summary and Status

print("\n" + "="*60)
print("MODEL TESTING COMPLETED!")
print("="*60)

print("\nFor Streamlit deployment:")
print("🔹 Same TF-IDF vectorizer for both models")
print("🔹 Switch between SVM, DT and AdaBoost classifiers")
print("🔹 Compare predictions from all models")
print("🔹 Show agreement/disagreement analysis")



MODEL TESTING COMPLETED!

For Streamlit deployment:
🔹 Same TF-IDF vectorizer for both models
🔹 Switch between SVM, DT and AdaBoost classifiers
🔹 Compare predictions from all models
🔹 Show agreement/disagreement analysis


In [ ]:
#Create Model Summary by comparing all models

print("MODEL SUMMARY FOR DEPLOYMENT")
print("=" * 50)

print(f"Best performing model: {svm_best_pipeline}")
print(f"Final accuracy: {pipeline_accuracy:.4f}")
print(f"Cross-validation score: {svm_cv_scores.mean():.4f} (+/- {svm_cv_scores.std() * 2:.4f})")

print(f"\nTraining data statistics:")
print(f"- Total samples: {len(X)}")
print(f"- Human samples: {sum(y)}")
print(f"- AI samples: {len(y) - sum(y)}")

print(f"\nModel features:")
print(f"- Text preprocessing: Advanced with lemmatization")
print(f"- Vectorization: TF-IDF with {tfidf_vectorizer.max_features} features")
print(f"- Algorithm: {svm_best_pipeline}")

print(f"\nFiles created for Streamlit deployment:")
print(f"1. {pipeline_filename} - Complete ML pipeline")
print(f"2. {tfidf_filename} - TF-IDF vectorizer")

print(f"\nNext steps:")
print("1. Downloaded the .pkl files from Colab")
print("2. We will use them in our Streamlit application")
print("3. The pipeline can process raw text and make predictions")

print("\Our ML models are ready for deployment!")

MODEL SUMMARY FOR DEPLOYMENT
Best performing model: Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_features=5000, ngram_range=(1, 3))),
                ('classifier',
                 SVC(C=10, kernel='linear', probability=True,
                     random_state=42))])
Final accuracy: 0.9692
Cross-validation score: 0.9732 (+/- 0.0138)

Training data statistics:
- Total samples: 3728
- Human samples: 1864
- AI samples: 1864

Model features:
- Text preprocessing: Advanced with lemmatization
- Vectorization: TF-IDF with 5000 features
- Algorithm: Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_features=5000, ngram_range=(1, 3))),
                ('classifier',
                 SVC(C=10, kernel='linear', probability=True,
                     random_state=42))])

Files created for Streamlit deployment:
1. human_ai_pipeline.pkl - Complete ML pipeline
2. tfidf_vectorizer.pkl - TF-IDF vectorizer

Next steps:
1. Downloaded the .pkl files from Colab
2. W